# PORTFOLIO OPTIMIZATION ON SOCIAL TRADING


## Il modello fa una ricerca dei pesi migliori che massimizzano una funzione. Tale funzione esprime la differenza dal rendimento atteso del portafoglio e la volatilità di quest'ultimo. Simula i vari pesi, che hanno determinati vincoli, e poi li testa sulla funzione. 

In [177]:
import pandas as pd       
import numpy as np 
import random
df = pd.read_csv('etoro.csv', sep=';')
df.index = df['data']
df = df.dropna(axis=1, how='all')
df = df.drop('data',axis=1).dropna()
df = df[-17:]
for col_i in col:
    df[col_i] = df[col_i].astype('float')
df = df.loc[:, (df != 0).any(axis=0)]
df = df.loc[:,~df.columns.duplicated()]
for coli in df.columns:
    if coli[-2:-1]=='.':
        df = df.drop(coli, 1)
col = list(df.columns)
print(len(df.columns))

df

49


,jaynemesis,openwings,slowestendy,Robert8E,smartandsocial,Ilakha,clahuz,vidinho,cristianan,richiardstroud,...,muchmanyCN,alliancia10,imbolex,valuefund,laijianming,zomorod,eric81723,dennyzee,valentinagrani,ismaelibrahim
data,,,,,,,,,,,,,,,,,,,,,
201901,12.14,141.28,0.00,-2.61,0.32,0.00,-1.22,-2.23,-0.36,12.28,...,0.00,9.83,5.34,5.55,0.00,18.82,0.00,-3.26,2.12,4.42
201902,7.13,-0.87,0.00,-1.02,0.71,0.00,0.81,-0.05,0.73,0.12,...,0.00,6.32,2.04,3.54,0.00,3.55,0.00,0.22,7.68,-1.33
201903,5.02,1.24,0.00,0.00,0.13,1.24,0.63,2.57,0.50,3.41,...,0.00,0.68,5.75,5.43,0.00,1.89,-3.37,-2.24,-0.09,1.07
201904,6.86,2.53,0.18,0.00,3.70,1.25,2.33,9.40,2.49,3.98,...,0.00,1.60,11.35,4.19,0.00,2.64,-0.84,4.69,4.70,-0.67
201905,17.68,-2.55,-2.33,0.74,-1.43,0.42,2.05,17.24,6.47,-5.37,...,0.00,4.06,3.23,-1.44,0.00,-14.50,-11.94,5.32,-8.63,-9.11
201906,4.17,3.44,4.37,8.05,-1.15,0.95,-0.42,1.77,1.65,9.03,...,1.86,2.45,5.54,4.25,-0.97,17.59,9.42,1.15,2.63,11.37
201907,-5.19,1.34,0.91,-0.89,1.93,-0.20,1.05,-2.35,-3.03,3.50,...,14.69,3.49,3.62,3.02,0.25,10.66,0.97,0.13,3.28,2.07
201908,-4.71,1.00,0.13,-0.84,6.14,3.45,-2.99,-2.20,-2.02,-4.99,...,4.16,0.40,-2.14,-3.06,-7.39,3.46,-7.83,3.35,0.23,-0.77
201909,-4.61,-0.60,0.02,-3.66,1.32,0.22,2.95,0.68,-1.21,-0.52,...,-3.58,-1.74,1.77,1.40,10.09,3.23,3.11,2.49,5.28,3.49


# NORMALITY TEST

In [178]:
from scipy import stats
np.random.seed(12345678)
x = stats.norm.rvs(loc=5, scale=3, size=100)

for col_n in range(0,len(col)):
    stat, p = stats.shapiro(df[col[col_n]])
    print(str(col[col_n])+': Statistics=%.3f, p=%.3f' % (stat, p))
    # interpret
    alpha = 0.05
    if p < alpha:
        print('Sample does not look Gaussian (reject H0)')
        df = df.drop(col[col_n], 1)
        
col = list(df.columns)
print(len(df.columns))

jaynemesis: Statistics=0.957, p=0.567
openwings: Statistics=0.296, p=0.000
Sample does not look Gaussian (reject H0)
slowestendy: Statistics=0.842, p=0.008
Sample does not look Gaussian (reject H0)
Robert8E: Statistics=0.783, p=0.001
Sample does not look Gaussian (reject H0)
smartandsocial: Statistics=0.938, p=0.290
Ilakha: Statistics=0.951, p=0.473
clahuz: Statistics=0.966, p=0.738
vidinho: Statistics=0.745, p=0.000
Sample does not look Gaussian (reject H0)
cristianan: Statistics=0.924, p=0.172
richiardstroud: Statistics=0.948, p=0.420
marek1198: Statistics=0.697, p=0.000
Sample does not look Gaussian (reject H0)
andrefranklin: Statistics=0.903, p=0.076
darekwa: Statistics=0.938, p=0.296
ghalex: Statistics=0.958, p=0.596
kubyk1975: Statistics=0.939, p=0.310
simonyung: Statistics=0.893, p=0.052
infinity0683: Statistics=0.966, p=0.738
2016dennis: Statistics=0.884, p=0.037
Sample does not look Gaussian (reject H0)
vasultoro: Statistics=0.979, p=0.945
tongschef: Statistics=0.933, p=0.241


In [179]:
df

,jaynemesis,smartandsocial,Ilakha,clahuz,cristianan,richiardstroud,andrefranklin,darekwa,ghalex,kubyk1975,...,oceantan007,hyjbrighter,cwhw1023,alliancia10,imbolex,valuefund,zomorod,eric81723,dennyzee,ismaelibrahim
data,,,,,,,,,,,,,,,,,,,,,
201901,12.14,0.32,0.00,-1.22,-0.36,12.28,6.47,14.41,11.68,-0.78,...,15.54,0.00,6.53,9.83,5.34,5.55,18.82,0.00,-3.26,4.42
201902,7.13,0.71,0.00,0.81,0.73,0.12,1.41,4.51,0.73,-4.65,...,3.08,0.00,6.10,6.32,2.04,3.54,3.55,0.00,0.22,-1.33
201903,5.02,0.13,1.24,0.63,0.50,3.41,1.14,4.13,2.87,-0.19,...,5.04,0.00,0.22,0.68,5.75,5.43,1.89,-3.37,-2.24,1.07
201904,6.86,3.70,1.25,2.33,2.49,3.98,6.10,5.80,3.95,2.98,...,5.33,0.00,-1.84,1.60,11.35,4.19,2.64,-0.84,4.69,-0.67
201905,17.68,-1.43,0.42,2.05,6.47,-5.37,-4.71,-8.23,-8.69,2.31,...,-9.66,-1.69,-12.14,4.06,3.23,-1.44,-14.50,-11.94,5.32,-9.11
201906,4.17,-1.15,0.95,-0.42,1.65,9.03,2.73,5.24,9.18,-1.95,...,7.87,5.14,12.95,2.45,5.54,4.25,17.59,9.42,1.15,11.37
201907,-5.19,1.93,-0.20,1.05,-3.03,3.50,1.20,4.05,1.21,-1.41,...,1.91,3.06,3.93,3.49,3.62,3.02,10.66,0.97,0.13,2.07
201908,-4.71,6.14,3.45,-2.99,-2.02,-4.99,-0.79,-3.08,-2.63,2.82,...,0.36,-6.29,-3.39,0.40,-2.14,-3.06,3.46,-7.83,3.35,-0.77
201909,-4.61,1.32,0.22,2.95,-1.21,-0.52,-0.13,1.19,-0.27,-2.09,...,-2.75,3.32,1.60,-1.74,1.77,1.40,3.23,3.11,2.49,3.49


In [235]:
def f(df,k):   # funzione da ottimizzare
    var=df.var()**0.5 # sigma_i
    dff=df.corr()*var # sigma_i*corr_ij
    dff2=dff.copy()
    for coli in col:
        dff2[coli]=dff[coli]*var*k # sigma_i*corr_ii*sigma_j*w_j
    dff2=dff2*k   # sigma_i*corr_ii*sigma_j*w_j*w_i
    #np.fill_diagonal(dff2.values, 0)
    df3=dff2.where(np.tril(np.ones(dff2.shape)).astype(np.bool))

    df3=df3.fillna(0)
    somma1 =(df.mean()*k).sum()
    somma2=sum(var**2*k)+df3.sum().sum()
    if somma2==0:
        somma2=1
    return somma1,somma2,somma1 / somma2

#check sul massimo
def check(pesimin, ris, somma1, somma2, valmin, file, capitale,k):
        if ris>valmin:
            pesimin=k
            valmin=ris
            rendatteso=somma1
            rischioatteso=somma2
            out=pd.DataFrame(pesimin[pesimin>0]*capitale)
            out['sim']=r
            out['ris']=ris
            out.to_csv(file)
            print('saved')
            return pesimin,valmin
        else:
            return pesimin, valmin
        


In [ ]:
capitale=5000
x=np.zeros(len(col))
pesimin=x
valmin=0
rendatteso=0
rischioatteso=0
peso=20
for r in range(0,1000000):
    y=[]
    tot=100
    for i in range(0,len(x)):
        a=random.randrange(0, peso, 2)
        if tot-a<=0:
            a=tot
            tot=0
        y.append(a/100)
        if tot!=0:
            tot=tot - a
    for j in range(0,50):
        random.shuffle(y)
        k=pd.DataFrame(y)
        k.index=col
        k.columns=['a']
        k=k['a']
        somma1, somma2, ris = f(df,k)
        pesimin, valmin = check(pesimin,ris, somma1, somma2, valmin, 'output.csv', capitale,k)

saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved


# Modello più performante per la scelta dei pesi (Gradient Ascent)

In [172]:
capitale=6760
x=np.zeros(len(col))
pesimin=x
valmin=0
rendatteso=0
rischioatteso=0
from scipy.special import softmax

def dw(df,k):
        var=df.var()**0.5
        dff=df.corr()*var
        dff2=dff.copy()
        for coli in col:
            dff2[coli]=dff[coli]*var*k
        df3=dff2.where(np.tril(np.ones(dff2.shape)).astype(np.bool))

        df3=df3.fillna(0)
        somma1=df.mean() 
        somma2= 2*k*df.var()**0.5**2- df3.sum()
        return somma1, somma2
    


for r in range(0,1):
    tot=100+5
    y=[]
    for i in range(0,len(x)):
        a=random.randrange(0, tot, 5) 
        y.append(a/100)
        tot=tot - a
    for j in range(0,50):
        random.shuffle(y)
        k=pd.DataFrame(y)
        k.index=col
        k.columns=['a']
        k=k['a']

        
somma1, somma2, ris = f(df,k)
print(ris)
for g in range(0,100):   

        somma1, somma2 = dw(df,k)
        k = k + 0.05*(somma1 -somma2)
        somma1, somma2, ris = f(df,k)
        print(ris)
val = softmax(k.values) 
k=pd.DataFrame(val)
k.index=col
k.columns=['a']
k=k['a']
k

-65.58944707857141
-4988.514688321605
-1524782.7763059302
-1457071454.754629
-1893001377827.4548
-2595473446325217.0
-3.5976375785688207e+18
-4.99817455419184e+21
-6.946094321488778e+24
-9.652178440696623e+27
-1.3410578073880165e+31
-1.863037072400491e+34
-2.587993396424805e+37
-3.5948781905360413e+40
-4.993353040635307e+43
-6.93573280006671e+46
-9.633575082928596e+49
-1.338072268923358e+53
-1.8585308577308965e+56
-2.5814209282453712e+59
-3.5854788590499467e+62
-4.9800656529559095e+65
-6.917078447855427e+68
-9.607495031591875e+71
-1.3344353376522085e+75
-1.8534669106892556e+78
-2.5743767324951226e+81
-3.575685738414697e+84
-4.966455728632488e+87
-6.8981682954474345e+90
-9.581224081727253e+93
-1.3307859457088212e+97
-1.8483976678553162e+100
-2.567335445231942e+103
-3.565905427704242e+106
-4.952871090034667e+109
-6.879299665953294e+112
-9.555016275141879e+115
-1.3271457913995752e+119
-1.8433416549271925e+122
-2.560312874798604e+125
-3.556151405215943e+128
-4.939323213512686e+131
-6.86048

jaynemesis        0.0
openwings         1.0
slowestendy       0.0
Robert8E          0.0
smartandsocial    0.0
Ilakha            0.0
clahuz            0.0
vidinho           0.0
cristianan        0.0
richiardstroud    0.0
marek1198         0.0
andrefranklin     0.0
darekwa           0.0
ghalex            0.0
kubyk1975         0.0
simonyung         0.0
infinity0683      0.0
2016dennis        0.0
mimihaha          0.0
vasultoro         0.0
tongschef         0.0
misterg23         0.0
richardstroud     0.0
rayvahey          0.0
johnliontw        0.0
daniel4653        0.0
jakey69           0.0
marek1198.1       0.0
harshmit          0.0
mimihaha.1        0.0
socialinvestor    0.0
stockssqueezer    0.0
jdominguez        0.0
clicktrader       0.0
kubyk1975.1       0.0
Benschi87         0.0
avishaib          0.0
swisscashcow      0.0
kingpdef          0.0
Name: a, dtype: float64

# Modello più performante per la scelta dei pesi (Gradient Ascent con tensorflow)


In [57]:
import tensorflow as tf

x = tf.Variable(2, name = 'x', dtype = tf.float32)
f_x = tf.log(x)

y = tf.Variable(5, name = 'y', dtype = tf.float32)


f_xy= tf.square(tf.log(x)) / tf.pow(y,3)
#f_xy =tf.pow(2*x+2,2)
optimizer = tf.train.GradientDescentOptimizer(0.5)
train = optimizer.minimize(f_xy)

init = tf.initialize_all_variables()

def optimize():
   with tf.Session() as session:
      session.run(init)
      print("starting at", "x:", session.run(x), "y:", session.run(y), "sqrt(log(x)) / y^3 :", session.run(f_xy))
      
      for step in range(1000):
         session.run(train)
      print("starting at", "x:", session.run(x), "y:", session.run(y), "sqrt(log(x)) / y^3 :", session.run(f_xy))
def optimize():
   with tf.Session() as session:
      session.run(init)
      print("starting at", "x:", session.run(x), "y:", session.run(y), "sqrt(log(x)) / y^3 :", session.run(f_xy))
      
      for step in range(1000):
         session.run(train)
      print("starting at", "x:", session.run(x), "y:", session.run(y), "sqrt(log(x)) / y^3 :", session.run(f_xy))

optimize()

starting at x: 2.0 y: 5.0 sqrt(log(x)) / y^3 : 0.0038436241
starting at x: 1.0037093 y: 5.187768 sqrt(log(x)) / y^3 : 9.818331e-08


0.0769129257887224
